In [1]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt

In [2]:
(x_train, y_train), (x_test, y_test)=cifar10.load_data()

# 图像数据的转换:

In [16]:
# import glob
from PIL import Image
import numpy as np

input_path="test/3.jpg"
output_path="test/3_out_sm.jpg"
im=Image.open(input_path)
# im.show()

im.thumbnail((28,28))
print(im.format, im.size, im.mode)
im.save(output_path)
# im.show()

im_1=im.convert("L")
print("im_1",im_1.format, im_1.size, im_1.mode) 
im_1.save("test/3_out_gray.bmp")
im_data=im_1.getdata()
im_array = np.array(list(im_data))
im_array= im_array.reshape(28,28)
im_array=255-im_array
im_array=im_array.reshape(1,28,28,1)
print(im_array)

JPEG (28, 28) RGB
im_1 None (28, 28) L
[[[[ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]]

  [[ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]]

  [[ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]]

  [[ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]]

  [[ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
   [ 0]
 

In [4]:
import matplotlib
im = matplotlib.image.imread('test/3_out_gray.bmp')
print (im.shape)
print(im)

AttributeError: read